## AI & Machine Learning for Data Quality
**Description**: AI and machine learning can automate and enhance data quality checks by learning patterns and identifying anomalies more effectively than static rules.

**Task 1**: Training a model to predict and flag unusual trend patterns in sales data that
deviate from historical norms.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

def train_anomaly_detection_model(sales_data: pd.DataFrame, time_column: str, sales_column: str, contamination: float = 0.05):
    """
    Trains an Isolation Forest model to detect unusual trend patterns in sales data.

    Args:
        sales_data (pd.DataFrame): DataFrame with sales data, including a time column and a sales column.
        time_column (str): Name of the time column.
        sales_column (str): Name of the sales column.
        contamination (float, optional): The proportion of outliers expected in the dataset. Defaults to 0.05.

    Returns:
        tuple: A tuple containing the trained Isolation Forest model and the scaled sales data.
    """
    if time_column not in sales_data.columns or sales_column not in sales_data.columns:
        raise ValueError("Time column or sales column not found in DataFrame.")

    # Sort data by time
    sales_data = sales_data.sort_values(by=time_column).reset_index(drop=True)

    # Scale the sales data
    scaler = StandardScaler()
    scaled_sales = scaler.fit_transform(sales_data[[sales_column]])

    # Train Isolation Forest model
    model = IsolationForest(contamination=contamination, random_state=42)
    model.fit(scaled_sales)

    return model, scaler

def predict_unusual_trends(sales_data: pd.DataFrame, model: IsolationForest, scaler: StandardScaler, sales_column: str):
    """
    Predicts unusual trend patterns in new sales data using the trained Isolation Forest model.

    Args:
        sales_data (pd.DataFrame): DataFrame with new sales data.
        model (IsolationForest): Trained Isolation Forest model.
        scaler (StandardScaler): Fitted StandardScaler object.
        sales_column (str): Name of the sales column.

    Returns:
        pd.DataFrame: DataFrame with an additional column 'is_unusual_trend' indicating
                      whether a data point is predicted as an unusual trend (True) or not (False).
    """
    if sales_column not in sales_data.columns:
        raise ValueError("Sales column not found in DataFrame.")

    scaled_sales = scaler.transform(sales_data[[sales_column]])
    outlier_predictions = model.predict(scaled_sales)
    # Isolation Forest returns -1 for outliers and 1 for inliers
    sales_data['is_unusual_trend'] = np.where(outlier_predictions == -1, True, False)
    return sales_data

# Example Usage for Task 1:
# Generate some synthetic sales data with an unusual spike
np.random.seed(42)
dates = pd.to_datetime(pd.date_range(start='2024-01-01', end='2024-12-31', freq='D'))
normal_sales = 100 + 10 * np.sin(np.linspace(0, 10 * np.pi, len(dates))) + np.random.normal(0, 5, len(dates))
unusual_spike_index = len(dates) // 2
normal_sales[unusual_spike_index:unusual_spike_index+7] += 50  # Simulate a week-long spike
sales_df = pd.DataFrame({'date': dates, 'sales': normal_sales})

# Train the anomaly detection model
anomaly_model, sales_scaler = train_anomaly_detection_model(sales_df.copy(), time_column='date', sales_column='sales', contamination=0.02)

# Predict unusual trends on the same data
sales_df_with_predictions = predict_unusual_trends(sales_df.copy(), anomaly_model, sales_scaler, sales_column='sales')

# Visualize the results
plt.figure(figsize=(12, 6))
sns.lineplot(x='date', y='sales', data=sales_df_with_predictions, label='Sales')
unusual_trends = sales_df_with_predictions[sales_df_with_predictions['is_unusual_trend']]
sns.scatterplot(x='date', y='sales', data=unusual_trends, color='red', label='Unusual Trend')
plt.title('Sales Data with Unusual Trend Detection')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.legend()
plt.show()

print("\nTask 1: Unusual Trend Detection Completed")



NameError: name 'np' is not defined

**Task 2**: Using clustering algorithms to detect duplicate records where entries are not
exactly identical.

In [ ]:
# write your code from here
# Task 2 was implemented in the previous response. Please refer to that code block.

print("\nTask 2: Near Duplicate Detection using Clustering (Implementation in previous response)")

# Task 3: Implementing classification models to validate data based on learned characteristics from labeled datasets.
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

def train_data_validation_model(labeled_data: pd.DataFrame, feature_columns: list, target_column: str):
    """
    Trains a classification model to validate data based on learned characteristics.

    Args:
        labeled_data (pd.DataFrame): DataFrame with labeled data (valid/invalid).
        feature_columns (list): List of column names to use as features.
        target_column (str): Name of the column indicating data validity (e.g., 'is_valid').

    Returns:
        tuple: A tuple containing the trained classification model and the feature columns.
    """
    if not feature_columns or target_column not in labeled_data.columns or not all(col in labeled_data.columns for col in feature_columns):
        raise ValueError("Feature columns or target column not found in DataFrame.")

    X = labeled_data[feature_columns]
    y = labeled_data[target_column]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = RandomForestClassifier(random_state=42)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Data Validation Model Accuracy: {accuracy:.4f}")

    return model, feature_columns

def validate_new_data(new_data: pd.DataFrame, model: RandomForestClassifier, feature_columns: list):
    """
    Validates new data using the trained classification model.

    Args:
        new_data (pd.DataFrame): DataFrame with new data to validate.
        model (RandomForestClassifier): Trained classification model.
        feature_columns (list): List of feature columns used for training.

    Returns:
        pd.DataFrame: DataFrame with an additional column 'predicted_validity'
                      indicating the model's prediction of data validity (True/False).
    """
    if not all(col in new_data.columns for col in feature_columns):
        raise ValueError("Not all feature columns found in the new data.")

    X_new = new_data[feature_columns]
    predictions = model.predict(X_new)
    new_data['predicted_validity'] = predictions.astype(bool)
    return new_data

# Example Usage for Task 3:
# Create synthetic labeled data
np.random.seed(42)
data_labeled = {
    'feature1': np.random.rand(100),
    'feature2': np.random.rand(100),
    'error_code': np.random.choice(['A10', 'B25', 'C30', None], size=100),
    'value': np.random.randint(1, 100, size=100),
    'is_valid': [True] * 80 + [False] * 20  # Simulate some invalid records
}
df_labeled = pd.DataFrame(data_labeled)

# Define features and target
features = ['feature1', 'feature2', 'value']
target = 'is_valid'

# Train the data validation model
validation_model, model_features = train_data_validation_model(df_labeled.copy(), feature_columns=features, target_column=target)

# Create new data to validate
data_new = {
    'feature1': np.random.rand(20),
    'feature2': np.random.rand(20),
    'error_code': np.random.choice(['A10', 'B25', None, 'D40'], size=20),
    'value': np.random.randint(1, 100, size=20)
}
df_new = pd.DataFrame(data_new)

# Validate the new data
df_new_validated = validate_new_data(df_new.copy(), validation_model, feature_columns=model_features)
print("\nTask 3: Data Validation using Classification Model")
print(df_new_validated)


**Task 3**: Implementing classification models to validate data based on learned
characteristics from labeled datasets.

In [ ]:
# write your code from here

# Task 3: Implementing classification models to validate data based on learned characteristics from labeled datasets.
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

def train_data_validation_model(labeled_data: pd.DataFrame, feature_columns: list, target_column: str):
    """
    Trains a classification model to validate data based on learned characteristics.

    Args:
        labeled_data (pd.DataFrame): DataFrame with labeled data (valid/invalid).
        feature_columns (list): List of column names to use as features.
        target_column (str): Name of the column indicating data validity (e.g., 'is_valid').

    Returns:
        tuple: A tuple containing the trained classification model and the feature columns.
    """
    if not feature_columns or target_column not in labeled_data.columns or not all(col in labeled_data.columns for col in feature_columns):
        raise ValueError("Feature columns or target column not found in DataFrame.")

    X = labeled_data[feature_columns]
    y = labeled_data[target_column]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = RandomForestClassifier(random_state=42)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Data Validation Model Accuracy: {accuracy:.4f}")

    return model, feature_columns

def validate_new_data(new_data: pd.DataFrame, model: RandomForestClassifier, feature_columns: list):
    """
    Validates new data using the trained classification model.

    Args:
        new_data (pd.DataFrame): DataFrame with new data to validate.
        model (RandomForestClassifier): Trained classification model.
        feature_columns (list): List of feature columns used for training.

    Returns:
        pd.DataFrame: DataFrame with an additional column 'predicted_validity'
                      indicating the model's prediction of data validity (True/False).
    """
    if not all(col in new_data.columns for col in feature_columns):
        raise ValueError("Not all feature columns found in the new data.")

    X_new = new_data[feature_columns]
    predictions = model.predict(X_new)
    new_data['predicted_validity'] = predictions.astype(bool)
    return new_data

# Example Usage for Task 3:
# Create synthetic labeled data
np.random.seed(42)
data_labeled = {
    'feature1': np.random.rand(100),
    'feature2': np.random.rand(100),
    'error_code': np.random.choice(['A10', 'B25', 'C30', None], size=100),
    'value': np.random.randint(1, 100, size=100),
    'is_valid': [True] * 80 + [False] * 20  # Simulate some invalid records
}
df_labeled = pd.DataFrame(data_labeled)

# Define features and target
features = ['feature1', 'feature2', 'value']
target = 'is_valid'

# Train the data validation model
validation_model, model_features = train_data_validation_model(df_labeled.copy(), feature_columns=features, target_column=target)

# Create new data to validate
data_new = {
    'feature1': np.random.rand(20),
    'feature2': np.random.rand(20),
    'error_code': np.random.choice(['A10', 'B25', None, 'D40'], size=20),
    'value': np.random.randint(1, 100, size=20)
}
df_new = pd.DataFrame(data_new)

# Validate the new data
df_new_validated = validate_new_data(df_new.copy(), validation_model, feature_columns=model_features)
print("\nTask 3: Data Validation using Classification Model")
print(df_new_validated)
